# 🔴 Task 3 — English → French Translator (Encoder-Decoder Transformer)

---

### **Building on Day 8: From Decoder-Only GPT → Full Encoder-Decoder Transformer**

In Day 8 (`hands_on_part_7_scaling.ipynb`) we built a **decoder-only** GPT that generates Shakespeare. Now we extend it into an **encoder-decoder** Transformer for **machine translation**.

---

### **What Changes from Day 8?**

| Component | Day 8 (GPT) | Task 3 (Translator) | Status |
|-----------|-------------|---------------------|--------|
| `Head` | Self-attention only (Q,K,V from same input) | Accepts **optional external K,V** for cross-attention | 🔄 MODIFIED |
| `MultiHeadAttention` | Forwards single input `x` | Forwards **separate `x` and optional `context`** | 🔄 MODIFIED |
| `FeedForward` | Expand → ReLU → Contract → Dropout | **Identical** | ✅ UNCHANGED |
| `Block` (Decoder) | Self-Attention → FFN (2 sublayers) | Self-Attention → **Cross-Attention** → FFN (**3 sublayers**) | 🔄 MODIFIED |
| `EncoderBlock` | — | Self-Attention → FFN (**no causal mask**) | 🆕 NEW |
| `Encoder` | — | Embedding + Positional Enc + N × EncoderBlock | 🆕 NEW |
| `Decoder` | Was the whole `Transformer` class | Embedding + Positional Enc + N × DecoderBlock | 🔄 MODIFIED |
| `EncoderDecoderTransformer` | — | Encoder + Decoder + Linear head | 🆕 NEW |
| Data pipeline | Character-level, single text | **Word-level**, parallel EN↔FR pairs, `<PAD>` `<SOS>` `<EOS>` | 🆕 NEW |
| Generation | Autoregressive sampling | **Greedy decoding** with `<SOS>` → `<EOS>` | 🆕 NEW |

> **Legend:** ✅ UNCHANGED &nbsp;|&nbsp; 🔄 MODIFIED &nbsp;|&nbsp; 🆕 NEW

---

### **Architecture Overview**

```
English Input                        French Output
"I love you"                         "<SOS> Je t'aime <EOS>"
     ↓                                        ↓
[Token Embedding]                    [Token Embedding]
     ↓                                        ↓
[Positional Encoding]                [Positional Encoding]
     ↓                                        ↓
┌──────────────┐                    ┌──────────────────────┐
│   ENCODER    │                    │      DECODER         │
│              │                    │                      │
│ Self-Attn    │                    │ Masked Self-Attn     │
│ + Add & Norm │    enc_output      │ + Add & Norm         │
│              │ ──────────────►    │                      │
│ FFN          │                    │ Cross-Attn (Q=dec,   │
│ + Add & Norm │                    │   K,V=enc_output)    │
│              │                    │ + Add & Norm         │
│ × N layers   │                    │                      │
└──────────────┘                    │ FFN                  │
                                    │ + Add & Norm         │
                                    │                      │
                                    │ × N layers           │
                                    └──────────────────────┘
                                             ↓
                                    [Linear → Vocab Logits]
                                             ↓
                                    "Je t'aime"
```

---
## Section 1: Setup & Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import os
import zipfile
import urllib.request

---
## Section 2: Hyperparameters

🔄 **MODIFIED from Day 8** — Adjusted for a word-level translation task on CPU.

| Parameter | Day 8 (GPT) | Task 3 (Translator) | Why? |
|-----------|-------------|---------------------|------|
| `batch_size` | 32 | **64** | Shorter sequences, can fit more |
| `block_size` | 256 | **20** | Max sentence length (words, not chars) |
| `n_embed` | 384 | **128** | Smaller model for CPU training |
| `n_head` | 6 | **4** | Fewer heads for smaller d_model |
| `n_layer` | 6 | **3** | Fewer layers for tractable training |
| `dropout` | 0.2 | **0.1** | Less dropout for smaller model |
| `max_iters` | 5000 | **8000** | More iterations needed for translation |
| `learning_rate` | 3e-4 | **3e-4** | Same |

In [ ]:
# ──────────────────────────────────────────────
# Hyperparameters  (🔄 MODIFIED from Day 8)
# ──────────────────────────────────────────────
batch_size  = 64
block_size  = 20       # max sentence length in tokens (words)
max_iters   = 8000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters  = 100
n_embed     = 128      # embedding dimension
n_head      = 4        # number of attention heads
n_layer     = 3        # number of encoder/decoder blocks
dropout     = 0.1

print(f"Using device: {device}")

---
## Section 3: 🆕 Data Pipeline (NEW — Word-Level EN↔FR Pairs)

This is entirely new compared to Day 8's character-level Shakespeare pipeline.

We will:
1. Download English→French sentence pairs from Tatoeba / ManyThings.org
2. Clean and filter to short pairs (≤ `block_size` words)
3. Build word-level vocabularies with special tokens: `<PAD>`, `<SOS>`, `<EOS>`
4. Convert sentences to padded integer tensors

In [ ]:
# ──────────────────────────────────────────────
# 🆕 Download the dataset
# ──────────────────────────────────────────────
data_url  = "https://www.manythings.org/anki/fra-eng.zip"
zip_path  = "fra-eng.zip"
txt_file  = "fra.txt"

if not os.path.exists(txt_file):
    print("Downloading English-French dataset...")
    urllib.request.urlretrieve(data_url, zip_path)
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(".")
    print("✅ Download complete!")
else:
    print("✅ Dataset already exists.")

In [ ]:
# ──────────────────────────────────────────────
# 🆕 Load and clean sentence pairs
# ──────────────────────────────────────────────
import re
import unicodedata

def normalize_string(s):
    """Lowercase, strip accents (optional), and clean punctuation."""
    s = s.lower().strip()
    # Separate punctuation with spaces so they become their own tokens
    s = re.sub(r"([.!?])", r" \1", s)
    # Remove anything that isn't a letter, common punctuation, or space
    s = re.sub(r"[^a-zA-Zéèêëàâäùûüôöîïçœæ .!?'-]+", r" ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Read file — format: English\tFrench\tAttribution
pairs = []
with open(txt_file, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) >= 2:
            en = normalize_string(parts[0])
            fr = normalize_string(parts[1])
            pairs.append((en, fr))

print(f"Total pairs loaded: {len(pairs):,}")
print(f"\nExample pairs:")
for i in range(5):
    print(f"  EN: {pairs[i][0]}")
    print(f"  FR: {pairs[i][1]}")
    print()

In [ ]:
# ──────────────────────────────────────────────
# 🆕 Filter to short sentences (tractable on CPU)
# ──────────────────────────────────────────────
MAX_WORDS = block_size - 2  # leave room for <SOS> and <EOS>
NUM_PAIRS = 20000            # use first N pairs (already sorted shortest-first)

filtered_pairs = []
for en, fr in pairs[:50000]:  # search within first 50k
    en_words = en.split()
    fr_words = fr.split()
    if len(en_words) <= MAX_WORDS and len(fr_words) <= MAX_WORDS:
        filtered_pairs.append((en, fr))
    if len(filtered_pairs) >= NUM_PAIRS:
        break

# Shuffle for randomness
random.seed(42)
random.shuffle(filtered_pairs)

print(f"Filtered pairs: {len(filtered_pairs):,}")
print(f"Max words per sentence: {MAX_WORDS}")
print(f"\nSample filtered pairs:")
for i in range(5):
    print(f"  EN: {filtered_pairs[i][0]}")
    print(f"  FR: {filtered_pairs[i][1]}")
    print()

### 🆕 Building Word-Level Vocabularies

In Day 8 we had a **character-level** tokenizer (`stoi` / `itos`) built from all unique characters. Here we build a **word-level** tokenizer with special tokens.

| Special Token | Purpose |
|:---:|---|
| `<PAD>` (0) | Padding — fills shorter sentences to `block_size` |
| `<SOS>` (1) | Start-Of-Sequence — tells the decoder to start generating |
| `<EOS>` (2) | End-Of-Sequence — signals the sentence is complete |

In [ ]:
# ──────────────────────────────────────────────
# 🆕 Word-level tokenizer with special tokens
# ──────────────────────────────────────────────
# Compare with Day 8:
#   Day 8:  stoi = { ch: i for i, ch in enumerate(chars) }  ← character-level
#   Task 3: stoi = { word: i for i, word in enumerate(vocab) }  ← word-level + special tokens

PAD_TOKEN = "<PAD>"
SOS_TOKEN = "<SOS>"
EOS_TOKEN = "<EOS>"

def build_vocab(sentences):
    """Build word vocabulary from list of sentences."""
    word_counts = {}
    for sent in sentences:
        for word in sent.split():
            word_counts[word] = word_counts.get(word, 0) + 1
    
    # Special tokens first, then sorted vocabulary
    special_tokens = [PAD_TOKEN, SOS_TOKEN, EOS_TOKEN]
    vocab_words = special_tokens + sorted(word_counts.keys())
    
    stoi = {word: i for i, word in enumerate(vocab_words)}
    itos = {i: word for word, i in stoi.items()}
    return stoi, itos

# Build separate vocabularies for English and French
en_sentences = [p[0] for p in filtered_pairs]
fr_sentences = [p[1] for p in filtered_pairs]

en_stoi, en_itos = build_vocab(en_sentences)
fr_stoi, fr_itos = build_vocab(fr_sentences)

# Vocab sizes
src_vocab_size = len(en_stoi)
tgt_vocab_size = len(fr_stoi)

PAD_IDX = en_stoi[PAD_TOKEN]  # 0
SOS_IDX = en_stoi[SOS_TOKEN]  # 1
EOS_IDX = en_stoi[EOS_TOKEN]  # 2

print(f"English vocabulary size: {src_vocab_size:,}")
print(f"French vocabulary size:  {tgt_vocab_size:,}")
print(f"\nSpecial token indices: PAD={PAD_IDX}, SOS={SOS_IDX}, EOS={EOS_IDX}")
print(f"\nSample English tokens: {list(en_stoi.items())[:8]}")
print(f"Sample French tokens:  {list(fr_stoi.items())[:8]}")

In [ ]:
# ──────────────────────────────────────────────
# 🆕 Encode sentences to padded tensors
# ──────────────────────────────────────────────

def encode_sentence(sentence, stoi, max_len):
    """Encode a sentence to a list of token indices with <SOS>, <EOS>, and <PAD>."""
    words = sentence.split()
    tokens = [stoi[SOS_TOKEN]] + [stoi.get(w, stoi[PAD_TOKEN]) for w in words] + [stoi[EOS_TOKEN]]
    # Pad to max_len
    tokens = tokens + [stoi[PAD_TOKEN]] * (max_len - len(tokens))
    return tokens[:max_len]  # truncate if needed

def decode_sentence(token_ids, itos):
    """Decode token ids back to a sentence string."""
    words = []
    for idx in token_ids:
        word = itos[idx]
        if word == EOS_TOKEN:
            break
        if word not in (PAD_TOKEN, SOS_TOKEN):
            words.append(word)
    return " ".join(words)

# Encode all pairs
src_data = []  # English (encoder input)
tgt_data = []  # French  (decoder input/target)

for en, fr in filtered_pairs:
    src_data.append(encode_sentence(en, en_stoi, block_size))
    tgt_data.append(encode_sentence(fr, fr_stoi, block_size))

src_data = torch.tensor(src_data, dtype=torch.long)
tgt_data = torch.tensor(tgt_data, dtype=torch.long)

print(f"Source (EN) tensor shape: {src_data.shape}")
print(f"Target (FR) tensor shape: {tgt_data.shape}")

# Verify encoding/decoding
print(f"\nExample:")
print(f"  EN original: {filtered_pairs[0][0]}")
print(f"  EN encoded:  {src_data[0].tolist()}")
print(f"  EN decoded:  {decode_sentence(src_data[0].tolist(), en_itos)}")
print(f"  FR original: {filtered_pairs[0][1]}")
print(f"  FR encoded:  {tgt_data[0].tolist()}")
print(f"  FR decoded:  {decode_sentence(tgt_data[0].tolist(), fr_itos)}")

In [ ]:
# ──────────────────────────────────────────────
# 🆕 Train / Validation split
# ──────────────────────────────────────────────
# Day 8 used 90/10 split on characters. We do the same on sentence pairs.

n = int(len(src_data) * 0.9)
train_src, val_src = src_data[:n], src_data[n:]
train_tgt, val_tgt = tgt_data[:n], tgt_data[n:]

print(f"Training pairs:   {len(train_src):,}")
print(f"Validation pairs: {len(val_src):,}")

In [ ]:
# ──────────────────────────────────────────────
# 🔄 Batch loading (MODIFIED from Day 8)
# ──────────────────────────────────────────────
# Day 8 sampled random windows from one long text.
# Here we sample random (src, tgt) sentence pairs.

torch.manual_seed(1337)

def get_batch(split):
    """Sample a batch of (source, target) pairs."""
    if split == 'train':
        src, tgt = train_src, train_tgt
    else:
        src, tgt = val_src, val_tgt
    
    ix = torch.randint(len(src), (batch_size,))
    src_batch = src[ix].to(device)  # (B, T)
    tgt_batch = tgt[ix].to(device)  # (B, T)
    return src_batch, tgt_batch

In [ ]:
# ──────────────────────────────────────────────
# 🔄 Loss estimation (MODIFIED from Day 8)
# ──────────────────────────────────────────────
# Same structure as Day 8, but now we pass src and tgt separately,
# and ignore loss on <PAD> tokens.

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            src_batch, tgt_batch = get_batch(split)
            # Decoder input: all tokens except last
            # Decoder target: all tokens except first (<SOS>)
            dec_input = tgt_batch[:, :-1]
            dec_target = tgt_batch[:, 1:]
            logits = model(src_batch, dec_input)
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = dec_target.reshape(B * T)
            loss = F.cross_entropy(logits, targets, ignore_index=PAD_IDX)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

---
## Section 4: Model Components

We now build the model piece by piece, clearly marking what's changed from Day 8.

### 4.1 `Head` — Single Attention Head

🔄 **MODIFIED from Day 8**

**Day 8:** Q, K, V all come from the **same** input `x`.
**Task 3:** Q comes from `x`, but K and V can come from a **different** source (encoder output) — this is what makes **cross-attention** possible!

```python
# Day 8 (self-attention only):
def forward(self, x):
    k = self.key(x)        # K from x
    q = self.query(x)      # Q from x
    v = self.value(x)      # V from x

# Task 3 (self-attention OR cross-attention):
def forward(self, x, context=None):
    source = context if context is not None else x
    k = self.key(source)   # K from context (encoder output) or x
    q = self.query(x)      # Q always from x (decoder state)
    v = self.value(source)  # V from context (encoder output) or x
```

> **This is the KEY change** that enables the encoder-decoder architecture!

In [ ]:
# ──────────────────────────────────────────────
# Head  (🔄 MODIFIED — now supports cross-attention)
# ──────────────────────────────────────────────

class Head(nn.Module):
    """Single head of attention.
    
    🔄 CHANGE from Day 8:
       - Added `context` parameter to forward()
       - When context is None  → self-attention  (same as Day 8)
       - When context is given → cross-attention  (Q from x, K/V from context)
       - Causal mask only applied when use_causal_mask=True
    """

    def __init__(self, head_size, use_causal_mask=True):          # 🔄 NEW parameter
        super().__init__()
        self.key   = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.use_causal_mask = use_causal_mask                    # 🔄 NEW
        if use_causal_mask:
            self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, context=None):                           # 🔄 NEW parameter
        B, T, C = x.shape
        source = context if context is not None else x            # 🔄 NEW line
        
        q = self.query(x)                                         # Q always from x
        k = self.key(source)                                      # 🔄 K from source
        v = self.value(source)                                    # 🔄 V from source
        
        # Attention scores
        wei = q @ k.transpose(-2, -1) * C**-0.5
        
        # Causal mask only for decoder self-attention                🔄 CONDITIONAL
        if self.use_causal_mask:
            wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        out = wei @ v
        return out

### 4.2 `MultiHeadAttention`

🔄 **MODIFIED from Day 8** — Passes `context` through to each head.

```python
# Day 8:
def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)

# Task 3:
def forward(self, x, context=None):         # 🔄 Added context parameter
    out = torch.cat([h(x, context) for h in self.heads], dim=-1)  # 🔄 Pass context
```

In [ ]:
# ──────────────────────────────────────────────
# MultiHeadAttention  (🔄 MODIFIED — passes context)
# ──────────────────────────────────────────────

class MultiHeadAttention(nn.Module):
    """Multiple heads of attention in parallel.
    
    🔄 CHANGE from Day 8: Passes optional `context` to each Head.
    """

    def __init__(self, num_heads, head_size, use_causal_mask=True):  # 🔄 NEW param
        super().__init__()
        self.heads = nn.ModuleList([
            Head(head_size, use_causal_mask=use_causal_mask)        # 🔄 pass mask flag
            for _ in range(num_heads)
        ])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, context=None):                              # 🔄 NEW parameter
        out = torch.cat([h(x, context) for h in self.heads], dim=-1) # 🔄 pass context
        out = self.dropout(self.proj(out))
        return out

### 4.3 `FeedForward`

✅ **UNCHANGED from Day 8** — Exactly the same expand → ReLU → contract → dropout.

In [ ]:
# ──────────────────────────────────────────────
# FeedForward  (✅ UNCHANGED from Day 8)
# ──────────────────────────────────────────────

class FeedForward(nn.Module):
    """Position-wise feed-forward network — identical to Day 8."""

    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, n_embed * 4),
            nn.ReLU(),
            nn.Linear(n_embed * 4, n_embed),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

### 4.4 `EncoderBlock` 🆕 NEW

The Encoder block is simpler than the Decoder block — it has **no causal mask** and **no cross-attention**:

```
    ┌──────────────────────────┐
    │     Self-Attention       │  ← Bidirectional (no mask!)
    │     (see ALL tokens)     │
    ├──────────────────────────┤
    │     Add & LayerNorm      │
    ├──────────────────────────┤
    │     Feed-Forward         │
    ├──────────────────────────┤
    │     Add & LayerNorm      │
    └──────────────────────────┘
```

Compare with **Day 8's `Block`** which always used a causal mask. The Encoder block is the same structure but allows each token to attend to **all** other tokens (including future ones), since we want to fully understand the input.

In [ ]:
# ──────────────────────────────────────────────
# EncoderBlock  (🆕 NEW — no causal mask)
# ──────────────────────────────────────────────

class EncoderBlock(nn.Module):
    """Encoder block: Self-Attention (bidirectional) + FFN.
    
    🆕 This is like Day 8's Block, but with use_causal_mask=False
    so every token can attend to every other token — no masking!
    
    Day 8 Block had:    self.sa = MultiHeadAttention(n_head, head_size)  ← causal mask ON
    EncoderBlock has:   self.sa = MultiHeadAttention(n_head, head_size, use_causal_mask=False)
    """

    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa   = MultiHeadAttention(n_head, head_size, use_causal_mask=False)  # 🆕 No mask
        self.ffwd = FeedForward(n_embed)
        self.ln1  = nn.LayerNorm(n_embed)
        self.ln2  = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))        # Self-attention + residual
        x = x + self.ffwd(self.ln2(x))      # FFN + residual
        return x

### 4.5 `DecoderBlock` 🔄 MODIFIED

The Decoder block has **3 sublayers** instead of Day 8's 2:

```
Day 8 Block (2 sublayers):           Task 3 DecoderBlock (3 sublayers):

┌────────────────────────┐          ┌────────────────────────┐
│ Masked Self-Attention  │          │ Masked Self-Attention  │  ← Same as Day 8
│ + Add & Norm           │          │ + Add & Norm           │
├────────────────────────┤          ├────────────────────────┤
│ Feed-Forward           │          │ Cross-Attention        │  ← 🆕 NEW sublayer!
│ + Add & Norm           │          │ (Q=decoder, K,V=encoder│
└────────────────────────┘          │  output)               │
                                    │ + Add & Norm           │
                                    ├────────────────────────┤
                                    │ Feed-Forward           │  ← Same as Day 8
                                    │ + Add & Norm           │
                                    └────────────────────────┘
```

> **The cross-attention sublayer is the bridge between encoder and decoder!**

In [ ]:
# ──────────────────────────────────────────────
# DecoderBlock  (🔄 MODIFIED — adds cross-attention sublayer)
# ──────────────────────────────────────────────

class DecoderBlock(nn.Module):
    """Decoder block: Masked Self-Attention + Cross-Attention + FFN.
    
    🔄 CHANGES from Day 8's Block:
       1. Self-attention still has causal mask (same as Day 8)
       2. 🆕 Added cross-attention sublayer (Q from decoder, K/V from encoder)
       3. 🆕 Added extra LayerNorm (ln2) for cross-attention
       4. forward() now takes `enc_output` parameter
    """

    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        
        # Sublayer 1: Masked self-attention (same as Day 8)
        self.sa = MultiHeadAttention(n_head, head_size, use_causal_mask=True)
        self.ln1 = nn.LayerNorm(n_embed)
        
        # Sublayer 2: 🆕 Cross-attention (no causal mask — attend to full encoder output)
        self.cross_attn = MultiHeadAttention(n_head, head_size, use_causal_mask=False)
        self.ln2 = nn.LayerNorm(n_embed)
        
        # Sublayer 3: Feed-forward (same as Day 8)
        self.ffwd = FeedForward(n_embed)
        self.ln3 = nn.LayerNorm(n_embed)

    def forward(self, x, enc_output):                            # 🔄 NEW parameter
        # Sublayer 1: Masked self-attention + residual (same as Day 8)
        x = x + self.sa(self.ln1(x))
        
        # Sublayer 2: 🆕 Cross-attention + residual
        # Q comes from decoder (x), K and V come from encoder output
        x = x + self.cross_attn(self.ln2(x), context=enc_output) # 🆕 This line is NEW!
        
        # Sublayer 3: FFN + residual (same as Day 8)
        x = x + self.ffwd(self.ln3(x))
        return x

### 4.6 🆕 Sinusoidal Positional Encoding

Day 8 used **learned** positional embeddings (`nn.Embedding(block_size, n_embed)`).

For the encoder-decoder, we use **sinusoidal** positional encoding (from the original "Attention Is All You Need" paper), as covered in `encoder.ipynb`:

$$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right), \quad PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

> Either approach works — we use sinusoidal here for variety and to match the original Transformer paper.

In [ ]:
# ──────────────────────────────────────────────
# PositionalEncoding  (🆕 NEW — sinusoidal, not learned)
# ──────────────────────────────────────────────
# Day 8 used: self.position_embedding_table = nn.Embedding(block_size, n_embed)
# Task 3 uses: sinusoidal encoding (no learnable parameters)

class PositionalEncoding(nn.Module):
    """Sinusoidal positional encoding from 'Attention Is All You Need'."""

    def __init__(self, d_model, max_len=5000, dropout_rate=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout_rate)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)  # even dimensions
        pe[:, 1::2] = torch.cos(position * div_term)   # odd dimensions
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: (B, T, d_model)
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

---
## Section 5: 🆕 Complete Encoder-Decoder Transformer

In Day 8, the entire model was a single `Transformer` class (decoder-only).

Now we split it into **three classes**: `Encoder`, `Decoder`, and `EncoderDecoderTransformer`.

### 5.1 `Encoder` 🆕 NEW

The Encoder reads the full English input and produces rich contextual representations.

```python
# Day 8 had NO encoder — it was all one Transformer class.
# Now the encoder is a separate component.
```

In [ ]:
# ──────────────────────────────────────────────
# Encoder  (🆕 NEW)
# ──────────────────────────────────────────────

class Encoder(nn.Module):
    """Transformer Encoder: Embedding + Positional Encoding + N × EncoderBlock.
    
    Takes source tokens (English) and produces contextual representations
    that the decoder will attend to via cross-attention.
    """

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_embed, padding_idx=PAD_IDX)
        self.pos_encoding = PositionalEncoding(n_embed, max_len=block_size, dropout_rate=dropout)
        self.blocks = nn.ModuleList([EncoderBlock(n_embed, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)

    def forward(self, src):
        # src: (B, T_src) — English token indices
        x = self.token_embedding(src)   # (B, T_src, n_embed)
        x = self.pos_encoding(x)        # (B, T_src, n_embed)
        for block in self.blocks:
            x = block(x)
        x = self.ln_f(x)
        return x  # (B, T_src, n_embed) — encoder output

### 5.2 `Decoder` 🔄 MODIFIED

Compare with Day 8's `Transformer` class:

```python
# Day 8 Transformer (decoder-only):
class Transformer(nn.Module):
    def __init__(self):
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, y=None):
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)                    # ← no encoder output needed
        ...

# Task 3 Decoder:
class Decoder(nn.Module):
    def __init__(self, vocab_size):
        self.token_embedding = nn.Embedding(vocab_size, n_embed, padding_idx=PAD_IDX)
        self.pos_encoding = PositionalEncoding(n_embed, ...)
        self.blocks = nn.ModuleList([DecoderBlock(...) for _ in range(n_layer)])  # 🔄 DecoderBlock
        self.ln_f = nn.LayerNorm(n_embed)

    def forward(self, tgt, enc_output):       # 🔄 takes enc_output!
        ...
        for block in self.blocks:
            x = block(x, enc_output)           # 🔄 pass enc_output to each block
        ...
```

In [ ]:
# ──────────────────────────────────────────────
# Decoder  (🔄 MODIFIED from Day 8's Transformer class)
# ──────────────────────────────────────────────

class Decoder(nn.Module):
    """Transformer Decoder: Embedding + Positional Encoding + N × DecoderBlock.
    
    🔄 CHANGES from Day 8:
       1. Uses DecoderBlock (3 sublayers) instead of Block (2 sublayers)
       2. forward() takes enc_output in addition to target tokens
       3. Passes enc_output to each DecoderBlock for cross-attention
       4. No lm_head here — that goes in the full model
    """

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_embed, padding_idx=PAD_IDX)
        self.pos_encoding = PositionalEncoding(n_embed, max_len=block_size, dropout_rate=dropout)
        self.blocks = nn.ModuleList([
            DecoderBlock(n_embed, n_head) for _ in range(n_layer)    # 🔄 DecoderBlock
        ])
        self.ln_f = nn.LayerNorm(n_embed)

    def forward(self, tgt, enc_output):                              # 🔄 NEW parameter
        # tgt: (B, T_tgt) — French token indices
        x = self.token_embedding(tgt)    # (B, T_tgt, n_embed)
        x = self.pos_encoding(x)         # (B, T_tgt, n_embed)
        for block in self.blocks:
            x = block(x, enc_output)      # 🔄 pass encoder output for cross-attention
        x = self.ln_f(x)
        return x  # (B, T_tgt, n_embed)

### 5.3 `EncoderDecoderTransformer` 🆕 NEW

This is the complete model that ties everything together.

```
English tokens ─► Encoder ─► enc_output ─┐
                                          │ (cross-attention)
French tokens  ─► Decoder ◄──────────────┘
                    │
                    ▼
              Linear Head ─► French vocab logits
```

In [ ]:
# ──────────────────────────────────────────────
# EncoderDecoderTransformer  (🆕 NEW)
# ──────────────────────────────────────────────

torch.manual_seed(1337)

class EncoderDecoderTransformer(nn.Module):
    """Complete Encoder-Decoder Transformer for sequence-to-sequence translation.
    
    🆕 This class has no equivalent in Day 8 — it combines the Encoder and Decoder
    and adds the final linear projection to target vocabulary logits.
    """

    def __init__(self, src_vocab_size, tgt_vocab_size):
        super().__init__()
        self.encoder = Encoder(src_vocab_size)
        self.decoder = Decoder(tgt_vocab_size)
        self.lm_head = nn.Linear(n_embed, tgt_vocab_size)   # project to French vocab

    def forward(self, src, tgt):
        """
        Args:
            src: (B, T_src) English token indices
            tgt: (B, T_tgt) French token indices (decoder input)
        Returns:
            logits: (B, T_tgt, tgt_vocab_size)
        """
        enc_output = self.encoder(src)            # (B, T_src, n_embed)
        dec_output = self.decoder(tgt, enc_output) # (B, T_tgt, n_embed)
        logits = self.lm_head(dec_output)          # (B, T_tgt, tgt_vocab_size)
        return logits

    def translate(self, src, max_len=None):
        """
        🆕 Greedy decoding: translate a source sentence to target.
        
        Compare with Day 8's generate() which started from empty context.
        Here we start from <SOS> and condition on the encoder output.
        
        Day 8:
            idx = torch.zeros((1,1))  # start from nothing
            logits, _ = self(idx)     # single forward pass
            
        Task 3:
            enc_output = self.encoder(src)      # encode English first
            tgt = [<SOS>]                        # start with <SOS>
            logits = self(src_dummy, tgt)        # use encoder output
        """
        if max_len is None:
            max_len = block_size
        
        self.eval()
        with torch.no_grad():
            enc_output = self.encoder(src)   # (1, T_src, n_embed)
            
            # Start with <SOS> token
            tgt_tokens = torch.full((1, 1), SOS_IDX, dtype=torch.long, device=device)
            
            for _ in range(max_len - 1):
                dec_output = self.decoder(tgt_tokens, enc_output)
                logits = self.lm_head(dec_output)
                
                # Greedy: pick the most likely next token
                next_token = logits[:, -1, :].argmax(dim=-1, keepdim=True)  # (1, 1)
                tgt_tokens = torch.cat([tgt_tokens, next_token], dim=1)
                
                # Stop if <EOS> is predicted
                if next_token.item() == EOS_IDX:
                    break
        
        return tgt_tokens[0].tolist()

print("✅ EncoderDecoderTransformer defined!")

In [ ]:
# ──────────────────────────────────────────────
# Instantiate the model
# ──────────────────────────────────────────────
model = EncoderDecoderTransformer(src_vocab_size, tgt_vocab_size)
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")
print(f"  Encoder: {sum(p.numel() for p in model.encoder.parameters()):,}")
print(f"  Decoder: {sum(p.numel() for p in model.decoder.parameters()):,}")
print(f"  LM Head: {sum(p.numel() for p in model.lm_head.parameters()):,}")

---
## Section 6: Training

🔄 **MODIFIED from Day 8** — The training loop structure is the same, but:
- We pass `(src, tgt_input)` instead of `(x, y)` from a single sequence
- The decoder input is `tgt[:, :-1]` (everything except last token)
- The target is `tgt[:, 1:]` (everything except `<SOS>`)
- We use `ignore_index=PAD_IDX` in cross-entropy to skip padding

In [ ]:
# ──────────────────────────────────────────────
# Training Loop  (🔄 MODIFIED from Day 8)
# ──────────────────────────────────────────────

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

train_losses = []
val_losses = []

for i in range(max_iters):

    # Evaluate periodically (same as Day 8)
    if i % eval_interval == 0:
        losses = estimate_loss()
        train_losses.append(losses['train'].item())
        val_losses.append(losses['val'].item())
        print(f"step {i:5d}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Sample a batch  (🔄 MODIFIED — now (src, tgt) pairs)
    src_batch, tgt_batch = get_batch('train')
    
    # 🔄 NEW: Split target into decoder input and target labels
    dec_input  = tgt_batch[:, :-1]   # <SOS> w1 w2 ... wN       (teacher forcing)
    dec_target = tgt_batch[:, 1:]    # w1 w2 ... wN <EOS>/<PAD>

    # Forward pass  (🔄 MODIFIED — pass src and dec_input)
    logits = model(src_batch, dec_input)

    # Compute loss  (🔄 MODIFIED — ignore PAD tokens)
    B, T, C = logits.shape
    logits = logits.view(B * T, C)
    targets = dec_target.reshape(B * T)
    loss = F.cross_entropy(logits, targets, ignore_index=PAD_IDX)

    # Backprop (same as Day 8)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad(set_to_none=True)

# Final evaluation
losses = estimate_loss()
print(f"\nFinal: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

In [ ]:
# ──────────────────────────────────────────────
# Plot training curves
# ──────────────────────────────────────────────
plt.figure(figsize=(10, 5))
steps = [i * eval_interval for i in range(len(train_losses))]
plt.plot(steps, train_losses, label='Train Loss', marker='o')
plt.plot(steps, val_losses, label='Val Loss', marker='s')
plt.xlabel('Training Step')
plt.ylabel('Loss')
plt.title('Training & Validation Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## Section 7: 🆕 Translation — The Moment of Truth!

In Day 8, we generated Shakespeare from scratch. Here we **translate** English sentences into French using greedy decoding.

**Day 8 generation:**
```python
context = torch.zeros((1, 1), dtype=torch.long)  # start from empty
out = model.generate(context, max_new_tokens=1000)
print(decode(out[0].tolist()))
```

**Task 3 translation:**
```python
src = encode_english("I love you")   # encode the English input
out = model.translate(src)            # greedy decode to French
print(decode_french(out))             # "je t'aime"
```

In [ ]:
# ──────────────────────────────────────────────
# 🆕 Translate sentences!
# ──────────────────────────────────────────────

def translate(english_sentence):
    """Translate an English sentence to French."""
    # Encode the English sentence
    en_clean = normalize_string(english_sentence)
    tokens = encode_sentence(en_clean, en_stoi, block_size)
    src = torch.tensor([tokens], dtype=torch.long, device=device)
    
    # Translate using greedy decoding
    output_ids = model.translate(src, max_len=block_size)
    
    # Decode the French output
    french = decode_sentence(output_ids, fr_itos)
    return french

# Test on training examples
print("=" * 60)
print("TRANSLATIONS FROM TRAINING DATA")
print("=" * 60)
for i in range(10):
    en, fr_expected = filtered_pairs[i]
    fr_predicted = translate(en)
    print(f"\n  EN:       {en}")
    print(f"  Expected: {fr_expected}")
    print(f"  Got:      {fr_predicted}")

In [ ]:
# ──────────────────────────────────────────────
# 🆕 Test on UNSEEN sentences from validation set
# ──────────────────────────────────────────────

print("=" * 60)
print("TRANSLATIONS FROM VALIDATION DATA (UNSEEN)")
print("=" * 60)

val_start = int(len(filtered_pairs) * 0.9)  # validation split
for i in range(val_start, min(val_start + 10, len(filtered_pairs))):
    en, fr_expected = filtered_pairs[i]
    fr_predicted = translate(en)
    print(f"\n  EN:       {en}")
    print(f"  Expected: {fr_expected}")
    print(f"  Got:      {fr_predicted}")

In [ ]:
# ──────────────────────────────────────────────
# 🆕 Try your own sentences!
# ──────────────────────────────────────────────

test_sentences = [
    "I am happy.",
    "He is a student.",
    "She is beautiful.",
    "We are here.",
    "It is cold.",
    "I love you.",
    "Thank you.",
    "Good morning.",
]

print("=" * 60)
print("CUSTOM TRANSLATIONS")
print("=" * 60)
for en in test_sentences:
    fr = translate(en)
    print(f"\n  EN: {en}")
    print(f"  FR: {fr}")

---
## Section 8: 🆕 Visualise Attention Weights

Let's peek inside the model to see what the cross-attention is learning — which English words does the decoder focus on when generating each French word?

In [ ]:
# ──────────────────────────────────────────────
# 🆕 Attention weight visualization
# ──────────────────────────────────────────────

def get_cross_attention_weights(english_sentence):
    """Extract cross-attention weights from the last decoder layer."""
    model.eval()
    en_clean = normalize_string(english_sentence)
    tokens = encode_sentence(en_clean, en_stoi, block_size)
    src = torch.tensor([tokens], dtype=torch.long, device=device)
    
    # Encode
    with torch.no_grad():
        enc_output = model.encoder(src)
    
    # Decode step by step, collecting attention
    tgt_tokens = torch.full((1, 1), SOS_IDX, dtype=torch.long, device=device)
    fr_words = []
    
    # We'll hook into the last decoder block's cross-attention
    attention_maps = []
    
    last_block = model.decoder.blocks[-1]
    
    # Register a hook on the cross-attention
    hook_weights = {}
    def hook_fn(module, input, output):
        # input[0] is x, but we want the attention weights
        # We'll compute them manually here
        pass
    
    with torch.no_grad():
        for step in range(block_size - 1):
            dec_output = model.decoder(tgt_tokens, enc_output)
            logits = model.lm_head(dec_output)
            next_token = logits[:, -1, :].argmax(dim=-1, keepdim=True)
            
            word = fr_itos.get(next_token.item(), "<UNK>")
            if word == EOS_TOKEN:
                break
            if word != PAD_TOKEN and word != SOS_TOKEN:
                fr_words.append(word)
            
            tgt_tokens = torch.cat([tgt_tokens, next_token], dim=1)
    
    # Now do one final pass to get cross-attention weights
    # We manually compute attention in the last block's cross-attention heads
    with torch.no_grad():
        # Get decoder state just before cross-attention in last block
        x = model.decoder.token_embedding(tgt_tokens)
        x = model.decoder.pos_encoding(x)
        
        for block in model.decoder.blocks[:-1]:
            x = block(x, enc_output)
        
        # Last block: self-attention first
        x_after_sa = x + last_block.sa(last_block.ln1(x))
        
        # Cross-attention: compute weights manually
        x_normed = last_block.ln2(x_after_sa)
        
        # Collect weights from each head
        head_weights = []
        for head in last_block.cross_attn.heads:
            q = head.query(x_normed)
            k = head.key(enc_output)
            C = q.shape[-1]
            wei = q @ k.transpose(-2, -1) * C**-0.5
            wei = F.softmax(wei, dim=-1)
            head_weights.append(wei[0].cpu().numpy())  # (T_dec, T_src)
    
    en_words = en_clean.split()
    return head_weights, en_words, fr_words

In [ ]:
# ──────────────────────────────────────────────
# 🆕 Plot cross-attention heatmap
# ──────────────────────────────────────────────

def plot_attention(english_sentence, head_idx=0):
    """Plot cross-attention weights for a translation."""
    head_weights, en_words, fr_words = get_cross_attention_weights(english_sentence)
    
    if not fr_words:
        print("No output generated.")
        return
    
    # Average across heads
    avg_weights = np.mean(head_weights, axis=0)
    
    # Trim to actual words (skip <SOS> in decoder, skip padding in encoder)
    n_fr = len(fr_words) + 1  # +1 for <SOS> offset
    n_en = len(en_words) + 2  # +2 for <SOS> and <EOS>
    
    # Plot average attention
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Average attention
    attn_slice = avg_weights[1:len(fr_words)+1, 1:len(en_words)+1]  # skip special tokens
    if attn_slice.shape[0] > 0 and attn_slice.shape[1] > 0:
        im = axes[0].imshow(attn_slice, cmap='viridis', aspect='auto')
        axes[0].set_xticks(range(len(en_words)))
        axes[0].set_xticklabels(en_words, rotation=45, ha='right')
        axes[0].set_yticks(range(len(fr_words)))
        axes[0].set_yticklabels(fr_words)
        axes[0].set_xlabel('English (Source)')
        axes[0].set_ylabel('French (Target)')
        axes[0].set_title(f'Average Cross-Attention')
        plt.colorbar(im, ax=axes[0])
    
    # Plot 2: Specific head
    head_attn = head_weights[head_idx][1:len(fr_words)+1, 1:len(en_words)+1]
    if head_attn.shape[0] > 0 and head_attn.shape[1] > 0:
        im2 = axes[1].imshow(head_attn, cmap='viridis', aspect='auto')
        axes[1].set_xticks(range(len(en_words)))
        axes[1].set_xticklabels(en_words, rotation=45, ha='right')
        axes[1].set_yticks(range(len(fr_words)))
        axes[1].set_yticklabels(fr_words)
        axes[1].set_xlabel('English (Source)')
        axes[1].set_ylabel('French (Target)')
        axes[1].set_title(f'Head {head_idx} Cross-Attention')
        plt.colorbar(im2, ax=axes[1])
    
    plt.suptitle(f'"{english_sentence}" → "{" ".join(fr_words)}"', fontsize=13)
    plt.tight_layout()
    plt.show()

# Visualize attention for a few sentences
plot_attention("I love you.")
plot_attention("He is a student.")
plot_attention("It is cold.")

---
## Section 9: Save / Load Model Weights

Same pattern as Day 8.

In [ ]:
# ──────────────────────────────────────────────
# Save model weights (same pattern as Day 8)
# ──────────────────────────────────────────────
torch.save(model.state_dict(), 'translator_weights.pth')
print("✅ Model weights saved to translator_weights.pth")

# To load later:
# model.load_state_dict(torch.load('translator_weights.pth', map_location=device))
# model.eval()

---
## Summary: Day 8 GPT vs Task 3 Encoder-Decoder

| Aspect | Day 8 GPT (Decoder-Only) | Task 3 Translator (Encoder-Decoder) |
|--------|--------------------------|--------------------------------------|
| **Task** | Generate Shakespeare text | Translate English → French |
| **Architecture** | Decoder only | Encoder + Decoder with cross-attention |
| **Tokenization** | Character-level (65 chars) | Word-level (~5K-10K words + special tokens) |
| **Attention** | Causal self-attention only | Self-attention + **cross-attention** |
| **Block sublayers** | 2 (self-attn + FFN) | Encoder: 2, Decoder: **3** (+ cross-attn) |
| **Positional encoding** | Learned embeddings | Sinusoidal |
| **Input** | Single text stream | Parallel (source, target) pairs |
| **Generation** | Autoregressive sampling | Greedy decoding with `<SOS>`→`<EOS>` |
| **Loss** | CrossEntropy on all tokens | CrossEntropy **ignoring** `<PAD>` tokens |

### Key Takeaway

The **only architectural addition** to go from a decoder-only GPT to a full encoder-decoder Transformer is:

1. **An Encoder** (same blocks as decoder, minus the causal mask)
2. **Cross-attention** in each decoder block (Q from decoder, K/V from encoder output)

Everything else — the attention mechanism, feed-forward networks, layer norms, residual connections — is **exactly the same** as what you built in Day 8! 🎉